In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from pathlib import Path
import random
import time
# Фіксуємо сид для відтворюваності
torch.manual_seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [ ]:
data_root = Path("./vegetables")
train_dir = data_root / "train"
val_dir   = data_root / "validation"
test_dir  = data_root / "test"

In [ ]:
import shutil
from pathlib import Path

for split in ["train", "validation", "test"]:
    ckpt_dir = Path("/content/vegetables") / split / ".ipynb_checkpoints"
    if ckpt_dir.exists():
        print("Видаляю", ckpt_dir)
        shutil.rmtree(ckpt_dir)
    else:
        print("Немає", ckpt_dir)

Видаляю /content/vegetables/train/.ipynb_checkpoints
Видаляю /content/vegetables/validation/.ipynb_checkpoints
Видаляю /content/vegetables/test/.ipynb_checkpoints


Перед подачею в нейронну мережу всі зображення були приведені до фіксованого розміру.
Для тренувальної вибірки застосовувалась аугментація у вигляді випадкового горизонтального віддзеркалення (модель бачить помідор зліва, помідор справа,. Архітектура ResNet18 була попередньо натренована на великому датасеті ImageNet. У даній роботі було виконано перенесення знань: останній класифікаційний шар було замінено відповідно до кількості класів у задачі, після чого модель була донавчена на власному наборі зображень овочів.

In [ ]:
# Трансформації для простої CNN
simple_transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

simple_transform_eval = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Трансформації для ResNet18 (під ImageNet)
resnet_transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

resnet_transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

Цей блок коду зчитує зображення з папок train/validation/test, автоматично створює мітки класів на основі назв директорій, застосовує необхідні перетворення, визначає кількість класів та формує DataLoader’и для пакетної подачі даних у нейронну мережу під час навчання, валідації та тестування.

In [ ]:
# Для простої CNN
train_dataset_simple = datasets.ImageFolder(train_dir, transform=simple_transform_train)
val_dataset_simple   = datasets.ImageFolder(val_dir,   transform=simple_transform_eval)
test_dataset_simple  = datasets.ImageFolder(test_dir,  transform=simple_transform_eval)

print("Класи (порядок):", train_dataset_simple.classes)
num_classes = len(train_dataset_simple.classes)
print("num_classes =", num_classes)

batch_size = 32 #скільки картинок модель обробляє за один крок навчання

train_loader_simple = DataLoader(train_dataset_simple, batch_size=batch_size, shuffle=True)
val_loader_simple   = DataLoader(val_dataset_simple,   batch_size=batch_size, shuffle=False)
test_loader_simple  = DataLoader(test_dataset_simple,  batch_size=batch_size, shuffle=False)

Класи (порядок): ['Broccoli', 'Cucumber', 'Potato', 'Tomato']
num_classes = 4


Для класифікації зображень була реалізована проста згорткова нейронна мережа з трьома згортковими шарами, що послідовно збільшують кількість каналів з 3 до 64. Після кожного згорткового шару застосовувалась функція активації ReLU та операція підвибірки MaxPooling. Для вирівнювання розмірів використовувався Adaptive Average Pooling. Класифікація здійснювалась за допомогою двох повнозвʼязних шарів із застосуванням Dropout. Для навчання використовувалась функція втрат CrossEntropyLoss та оптимізатор Adam. На вході згорткової мережі використано 3 канали, що відповідає RGB-зображенням. У першому згортковому шарі використано 16 фільтрів для виділення простих ознак, таких як краї та контури. У наступних шарах кількість каналів поступово збільшено до 32 та 64 для витягання більш складних і абстрактних ознак. Розмір ядра 3×3 обрано як стандартний та ефективний для аналізу локальних структур на зображенні.



In [ ]:
# Проста CNN (2–3 конв. шари + fully connected)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.features = nn.Sequential(
            # conv1: 3 -> 16
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 128x128 -> 64x64

            # conv2: 16 -> 32
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64x64 -> 32x32

            # conv3: 32 -> 64
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32 -> 16x16
        )

        # adaptive pooling, щоб не рахувати розміри вручну
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))  # 64 x 16x16 -> 64 x 4x4

        self.classifier = nn.Sequential(
            nn.Flatten(),               # 64 * 4 * 4 = 1024
            nn.Linear(64 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x
simple_model = SimpleCNN(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_simple = optim.Adam(simple_model.parameters(), lr=1e-3)


Навчання моделі здійснювалось батчами з використанням алгоритму зворотного поширення помилки. Для кожного батча виконувались прямий прохід, обчислення функції втрат, зворотний прохід та оновлення ваг за допомогою оптимізатора Adam. Для оцінки якості моделі на валідаційній та тестовій вибірках використовувався окремий цикл без обчислення градієнтів.

In [ ]:
# Універсальні функції train / evaluate
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

Навчання моделі здійснювалось протягом 4 епох (зменшено з 10 початкових, бо було перенавчання). Після кожної епохи обчислювались значення функції втрат та точності на тренувальній і валідаційній вибірках. Під час навчання простої згорткової нейронної мережі було досягнуто високої точності вже на перших епохах. Максимальна валідаційна точність 100% була отримана на 4-й епосі. Подальше навчання не призвело до істотного покращення результатів, а незначні коливання точності зумовлені малою кількістю зображень у валідаційній вибірці. Для подальшого аналізу результати зберігались у вигляді історії навчання. Також було виміряно загальний час навчання моделі.

In [ ]:
# Навчання простої CNN
num_epochs_simple = 4

history_simple = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": [],
}

start_time = time.time()

for epoch in range(num_epochs_simple):
    train_loss, train_acc = train_one_epoch(
        simple_model, train_loader_simple, optimizer_simple, criterion, device
    )
    val_loss, val_acc = evaluate(simple_model, val_loader_simple, criterion, device)

    history_simple["train_loss"].append(train_loss)
    history_simple["train_acc"].append(train_acc)
    history_simple["val_loss"].append(val_loss)
    history_simple["val_acc"].append(val_acc)

    print(
        f"[SimpleCNN] Epoch {epoch+1}/{num_epochs_simple} | "
        f"Train: loss={train_loss:.4f}, acc={train_acc:.4f} | "
        f"Val: loss={val_loss:.4f}, acc={val_acc:.4f}"
    )

time_simple = time.time() - start_time
print(f"Час навчання SimpleCNN: {time_simple:.1f} с")

[SimpleCNN] Epoch 1/4 | Train: loss=0.0481, acc=0.9900 | Val: loss=0.0262, acc=0.9938
[SimpleCNN] Epoch 2/4 | Train: loss=0.0294, acc=0.9888 | Val: loss=0.0243, acc=0.9938
[SimpleCNN] Epoch 3/4 | Train: loss=0.0822, acc=0.9712 | Val: loss=0.1072, acc=0.9625
[SimpleCNN] Epoch 4/4 | Train: loss=0.0752, acc=0.9762 | Val: loss=0.0210, acc=1.0000
Час навчання SimpleCNN: 47.1 с


Ми використовуємо вже готову, попередньо навчену на ImageNet модель ResNet18 як екстрактор ознак, заморожуємо всі її шари, замінюємо лише останній класифікаційний шар під кількість власних класів і навчаємо тільки цей новий шар для розпізнавання овочів.

In [ ]:
# Тестова точність простої CNN - перевіряємо як прауює моделька :)
test_loss_simple, test_acc_simple = evaluate(simple_model, test_loader_simple, criterion, device)
print(f"[SimpleCNN] Test: loss={test_loss_simple:.4f}, acc={test_acc_simple:.4f}")

[SimpleCNN] Test: loss=0.0584, acc=0.9688


Для реалізації transfer learning було використано архітектуру ResNet18 з попередньо навченими на датасеті ImageNet вагами. Всі згорткові шари були заморожені, а останній повнозвʼязний шар класифікатора замінено відповідно до кількості класів у задачі. Для навчання нового класифікатора використовувалась функція втрат CrossEntropyLoss та оптимізатор Adam. Ми використовуємо вже готову, попередньо навчену на ImageNet модель ResNet18 як екстрактор ознак, заморожуємо всі її шари, замінюємо лише останній класифікаційний шар під кількість власних класів і навчаємо тільки цей новий шар для розпізнавання овочів.

In [ ]:
#Завантаження ResNet18 + заморозка фіч
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Заморожуємо всі шари, крім останнього
for param in resnet18.parameters():
    param.requires_grad = False

# Замінюємо класифікатор під 4 класи
in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features, num_classes)

resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.Adam(resnet18.fc.parameters(), lr=1e-3)

Під час навчання ResNet18 було виконано донавчання (transfer learning): всі згорткові шари були заморожені, а навчання здійснювалось лише для нового класифікаційного шару, який був адаптований під 4 класи овочів.

In [ ]:
# DataLoaders для ResNet18
train_dataset_resnet = datasets.ImageFolder(
    train_dir,
    transform=resnet_transform_train
)

val_dataset_resnet = datasets.ImageFolder(
    val_dir,
    transform=resnet_transform_eval
)

test_dataset_resnet = datasets.ImageFolder(
    test_dir,
    transform=resnet_transform_eval
)

batch_size = 32

train_loader_resnet = DataLoader(
    train_dataset_resnet,
    batch_size=batch_size,
    shuffle=True
)

val_loader_resnet = DataLoader(
    val_dataset_resnet,
    batch_size=batch_size,
    shuffle=False
)

test_loader_resnet = DataLoader(
    test_dataset_resnet,
    batch_size=batch_size,
    shuffle=False
)

# Навчання ResNet18
num_epochs_resnet = 5  # зазвичай вистачає менше епох

history_resnet = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": [],
}

start_time = time.time()

for epoch in range(num_epochs_resnet):
    train_loss, train_acc = train_one_epoch(
        resnet18, train_loader_resnet, optimizer_resnet, criterion, device
    )
    val_loss, val_acc = evaluate(resnet18, val_loader_resnet, criterion, device)

    history_resnet["train_loss"].append(train_loss)
    history_resnet["train_acc"].append(train_acc)
    history_resnet["val_loss"].append(val_loss)
    history_resnet["val_acc"].append(val_acc)

    print(
        f"[ResNet18] Epoch {epoch+1}/{num_epochs_resnet} | "
        f"Train: loss={train_loss:.4f}, acc={train_acc:.4f} | "
        f"Val: loss={val_loss:.4f}, acc={val_acc:.4f}"
    )

time_resnet = time.time() - start_time
print(f"Час навчання ResNet18: {time_resnet:.1f} с")



[ResNet18] Epoch 1/5 | Train: loss=0.2499, acc=0.9663 | Val: loss=0.1279, acc=0.9875
[ResNet18] Epoch 2/5 | Train: loss=0.1483, acc=0.9900 | Val: loss=0.0824, acc=0.9938
[ResNet18] Epoch 3/5 | Train: loss=0.1007, acc=0.9938 | Val: loss=0.0589, acc=0.9938
[ResNet18] Epoch 4/5 | Train: loss=0.0811, acc=0.9912 | Val: loss=0.0455, acc=1.0000
[ResNet18] Epoch 5/5 | Train: loss=0.0663, acc=0.9938 | Val: loss=0.0388, acc=1.0000
Час навчання ResNet18: 450.8 с


Після завершення навчання модель ResNet18 була протестована на тестовій вибірці, яка не використовувалась під час тренування. Було отримано значення функції втрат та точності, що характеризують узагальнювальну здатність моделі.

In [ ]:
# Тестова точність ResNet18
test_loss_resnet, test_acc_resnet = evaluate(resnet18, test_loader_resnet, criterion, device)
print(f"[ResNet18] Test: loss={test_loss_resnet:.4f}, acc={test_acc_resnet:.4f}")

[ResNet18] Test: loss=0.0621, acc=0.9938


Обидві моделі виконують одну й ту саму задачу багатокласової класифікації зображень. Різниця між ними полягає у способі навчання: SimpleCNN навчалась з нуля на наявних даних, тоді як ResNet18 використовувала попередньо навчені на ImageNet ознаки та донавчалась лише на рівні класифікатора.

In [ ]:
# порівняння
print("=== Порівняння моделей ===")
print(f"SimpleCNN:  val_acc={history_simple['val_acc'][-1]:.4f}, "
      f"test_acc={test_acc_simple:.4f}, time={time_simple:.1f} c")

print(f"ResNet18:   val_acc={history_resnet['val_acc'][-1]:.4f}, "
      f"test_acc={test_acc_resnet:.4f}, time={time_resnet:.1f} c")


=== Порівняння моделей ===
SimpleCNN:  val_acc=1.0000, test_acc=0.9688, time=47.1 c
ResNet18:   val_acc=1.0000, test_acc=0.9938, time=450.8 c


Незважаючи на однакову точність на валідаційній вибірці, тестова точність SimpleCNN виявилась нижчою, ніж у ResNet18. Це пояснюється кращою здатністю попередньо навчених мереж до узагальнення. Також слід враховувати малий розмір тестової вибірки, що призводить до значних коливань метрики навіть при одній помилці.
0.9688 ≈ 39 правильних з 40 (1/40≈2.5% помилка, 39 з 40 правильних → 0.975);
0.9938 ≈ 40 з 40, але 1 трохи “не вписалась” через округлення батчів